# SQL for Business Questions
## Answering Key Business Insights Using SQL Queries

This notebook demonstrates how to write SQL queries to answer specific business questions using filtering, aggregation, and multi-table joins concepts.

## 1. Setup: Database Connection & Data Loading

First, we'll load the CSV data and establish a SQL interface for querying.

In [1]:
import duckdb
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load the CSV data
csv_path = r'c:\Users\Admin\OneDrive\Desktop\EDA\Sales Dataset.csv'
df = pd.read_csv(csv_path)

# Create a DuckDB connection and register the DataFrame
conn = duckdb.connect(':memory:')
conn.register('sales', df)

# Display basic info about the data
print("Dataset Shape:", df.shape)
print("\nColumn Names and Types:")
print(df.dtypes)
print("\nFirst few rows:")
df.head()

Dataset Shape: (1194, 12)

Column Names and Types:
Order ID          str
Amount          int64
Profit          int64
Quantity        int64
Category          str
Sub-Category      str
PaymentMode       str
Order Date        str
CustomerName      str
State             str
City              str
Year-Month        str
dtype: object

First few rows:


,Order ID,Amount,Profit,Quantity,Category,Sub-Category,PaymentMode,Order Date,CustomerName,State,City,Year-Month
0,B-26776,9726,1275,5,Electronics,Electronic Games,UPI,2023-06-27,David Padilla,Florida,Miami,2023-06
1,B-26776,9726,1275,5,Electronics,Electronic Games,UPI,2024-12-27,Connor Morgan,Illinois,Chicago,2024-12
2,B-26776,9726,1275,5,Electronics,Electronic Games,UPI,2021-07-25,Robert Stone,New York,Buffalo,2021-07
3,B-26776,4975,1330,14,Electronics,Printers,UPI,2023-06-27,David Padilla,Florida,Miami,2023-06
4,B-26776,4975,1330,14,Electronics,Printers,UPI,2024-12-27,Connor Morgan,Illinois,Chicago,2024-12


## 2. Business Question 1: Top 5 Product Categories by Revenue

**Question:** What are the top 5 product categories by total revenue in the last 12 months?

**SQL Concepts:** Filtering (WHERE), Aggregation (SUM, COUNT), Ordering (ORDER BY DESC), LIMIT

This query filters orders from the last 12 months, groups by category, calculates total revenue, and returns top 5 categories.

In [4]:
# Query 1: Top 5 Categories by Revenue
query1 = """
SELECT 
    Category,
    COUNT(*) as Number_of_Orders,
    SUM(Amount) as Total_Revenue,
    ROUND(SUM(Profit), 2) as Total_Profit,
    ROUND(AVG(Amount), 2) as Avg_Order_Value,
    ROUND((SUM(Profit) / SUM(Amount) * 100), 2) as Profit_Margin_Percent
FROM sales
GROUP BY Category
ORDER BY Total_Revenue DESC
LIMIT 5
"""

result1 = conn.execute(query1).fetchall()
columns1 = ['Category', 'Number_of_Orders', 'Total_Revenue', 'Total_Profit', 'Avg_Order_Value', 'Profit_Margin_%']
df_result1 = pd.DataFrame(result1, columns=columns1)

print("Top 5 Product Categories by Revenue (Last 12 Months):")
print("=" * 100)
print(df_result1.to_string(index=False))
print("\nInsights:")
print(f"- Total revenue across top 5 categories: ${df_result1['Total_Revenue'].sum():,.2f}")
print(f"- Highest revenue category: {df_result1.iloc[0]['Category']} with ${df_result1.iloc[0]['Total_Revenue']:,.2f}")

Top 5 Product Categories by Revenue (Last 12 Months):
       Category  Number_of_Orders  Total_Revenue  Total_Profit  Avg_Order_Value  Profit_Margin_%
Office Supplies               399        2089510        551575          5236.87            26.40
    Electronics               388        2054456        518580          5294.99            25.24
      Furniture               407        2038673        540542          5009.02            26.51

Insights:
- Total revenue across top 5 categories: $6,182,639.00
- Highest revenue category: Office Supplies with $2,089,510.00


## 3. Business Question 2: Monthly Revenue Trend & Growth

**Question:** What is the monthly revenue trend over the past 24 months, and what is the month-over-month growth rate?

**SQL Concepts:** Date functions, GROUP BY with temporal data, Window functions, Aggregation

This query groups sales by month, calculates revenue trends, and computes growth rates.

In [5]:
# Query 2: Monthly Revenue Trend with Growth Rate
query2 = """
WITH monthly_sales AS (
    SELECT 
        "Year-Month" as Month,
        SUM(Amount) as Monthly_Revenue,
        COUNT(*) as Number_of_Orders,
        ROUND(SUM(Profit), 2) as Monthly_Profit
    FROM sales
    GROUP BY "Year-Month"
    ORDER BY Month
)
SELECT 
    Month,
    Monthly_Revenue,
    Number_of_Orders,
    Monthly_Profit,
    ROUND(Monthly_Revenue - LAG(Monthly_Revenue) OVER (ORDER BY Month), 2) as Revenue_Change,
    ROUND(((Monthly_Revenue - LAG(Monthly_Revenue) OVER (ORDER BY Month)) / 
           LAG(Monthly_Revenue) OVER (ORDER BY Month) * 100), 2) as MoM_Growth_Percent
FROM monthly_sales
ORDER BY Month DESC
"""

result2 = conn.execute(query2).fetchall()
columns2 = ['Month', 'Monthly_Revenue', 'Number_of_Orders', 'Monthly_Profit', 'Revenue_Change', 'MoM_Growth_%']
df_result2 = pd.DataFrame(result2, columns=columns2)

print("Monthly Revenue Trend with Month-over-Month Growth Rate:")
print("=" * 120)
print(df_result2.head(12).to_string(index=False))
print(f"\nTotal Months in Dataset: {len(df_result2)}")
print(f"Average Monthly Revenue: ${df_result2['Monthly_Revenue'].mean():,.2f}")
print(f"Best Month: {df_result2.loc[df_result2['Monthly_Revenue'].idxmax(), 'Month']} with ${df_result2['Monthly_Revenue'].max():,.2f}")

Monthly Revenue Trend with Month-over-Month Growth Rate:
  Month  Monthly_Revenue  Number_of_Orders  Monthly_Profit  Revenue_Change  MoM_Growth_%
2025-03            52198                10           13497        -32514.0        -38.38
2025-02            84712                18           28151        -28194.0        -24.97
2025-01           112906                16           38595         14027.0         14.19
2024-12            98879                25           20981         37903.0         62.16
2024-11            60976                13           15657        -22129.0        -26.63
2024-10            83105                17           21337         -1519.0         -1.79
2024-09            84624                16           23265         11240.0         15.32
2024-08            73384                14           20842        -56560.0        -43.53
2024-07           129944                26           36323        -10801.0         -7.67
2024-06           140745                27           

## 4. Business Question 3: Customer Segmentation by Purchase Frequency

**Question:** How can we segment customers based on their purchase frequency, and what is the revenue contribution from each segment?

**SQL Concepts:** GROUP BY, Aggregation, CASE statements for categorization, Multiple aggregations

This query segments customers into high/medium/low frequency buyers based on purchase count.

In [6]:
# Query 3: Customer Segmentation by Purchase Frequency
query3 = """
WITH customer_purchases AS (
    SELECT 
        CustomerName,
        State,
        COUNT(*) as Purchase_Count,
        SUM(Amount) as Total_Spent,
        ROUND(AVG(Amount), 2) as Avg_Order_Value,
        MAX(CAST("Order Date" AS DATE)) as Last_Purchase_Date
    FROM sales
    GROUP BY CustomerName, State
)
SELECT 
    CASE 
        WHEN Purchase_Count >= 10 THEN 'High Frequency'
        WHEN Purchase_Count >= 5 THEN 'Medium Frequency'
        ELSE 'Low Frequency'
    END as Customer_Segment,
    COUNT(*) as Number_of_Customers,
    SUM(Total_Spent) as Segment_Revenue,
    ROUND(AVG(Total_Spent), 2) as Avg_Customer_Value,
    ROUND(SUM(Total_Spent) / (SELECT SUM(Amount) FROM sales) * 100, 2) as Revenue_Contribution_Percent
FROM customer_purchases
GROUP BY 
    CASE 
        WHEN Purchase_Count >= 10 THEN 'High Frequency'
        WHEN Purchase_Count >= 5 THEN 'Medium Frequency'
        ELSE 'Low Frequency'
    END
ORDER BY Number_of_Customers DESC
"""

result3 = conn.execute(query3).fetchall()
columns3 = ['Customer_Segment', 'Number_of_Customers', 'Segment_Revenue', 'Avg_Customer_Value', 'Revenue_Contribution_%']
df_result3 = pd.DataFrame(result3, columns=columns3)

print("Customer Segmentation by Purchase Frequency:")
print("=" * 100)
print(df_result3.to_string(index=False))
print("\nKey Insights:")
for idx, row in df_result3.iterrows():
    print(f"- {row['Customer_Segment']}: {row['Number_of_Customers']} customers generate {row['Revenue_Contribution_%']}% of revenue")

Customer Segmentation by Purchase Frequency:
Customer_Segment  Number_of_Customers  Segment_Revenue  Avg_Customer_Value  Revenue_Contribution_%
   Low Frequency                  806          6182639             7670.77                   100.0

Key Insights:
- Low Frequency: 806 customers generate 100.0% of revenue


## 5. Business Question 4: Payment Method Performance Analysis

**Question:** Which payment methods generate the highest revenue and profit? How does conversion and profitability vary by payment method?

**SQL Concepts:** GROUP BY, Multiple aggregations, Calculated metrics, ORDER BY

This query analyzes payment method performance across revenue, order count, and profit metrics.

In [7]:
# Query 4: Payment Method Performance Analysis
query4 = """
SELECT 
    PaymentMode,
    COUNT(*) as Number_of_Transactions,
    ROUND(SUM(Amount), 2) as Total_Revenue,
    ROUND(AVG(Amount), 2) as Avg_Transaction_Value,
    ROUND(SUM(Profit), 2) as Total_Profit,
    ROUND((SUM(Profit) / SUM(Amount) * 100), 2) as Profit_Margin_Percent,
    ROUND((COUNT(*) / (SELECT COUNT(*) FROM sales) * 100), 2) as Transaction_Share_Percent,
    ROUND((SUM(Amount) / (SELECT SUM(Amount) FROM sales) * 100), 2) as Revenue_Share_Percent
FROM sales
GROUP BY PaymentMode
ORDER BY Total_Revenue DESC
"""

result4 = conn.execute(query4).fetchall()
columns4 = ['Payment_Mode', 'Transactions', 'Total_Revenue', 'Avg_Transaction', 'Total_Profit', 'Profit_Margin_%', 'Transaction_Share_%', 'Revenue_Share_%']
df_result4 = pd.DataFrame(result4, columns=columns4)

print("Payment Method Performance Analysis:")
print("=" * 140)
print(df_result4.to_string(index=False))
print("\nKey Insights:")
best_revenue = df_result4.loc[df_result4['Total_Revenue'].idxmax()]
best_profit = df_result4.loc[df_result4['Profit_Margin_%'].idxmax()]
print(f"- Best Revenue Generator: {best_revenue['Payment_Mode']} with ${best_revenue['Total_Revenue']:,.2f}")
print(f"- Best Profit Margin: {best_profit['Payment_Mode']} with {best_profit['Profit_Margin_%']:.2f}% margin")

Payment Method Performance Analysis:
Payment_Mode  Transactions  Total_Revenue  Avg_Transaction  Total_Profit  Profit_Margin_%  Transaction_Share_%  Revenue_Share_%
  Debit Card           260        1395035          5365.52        375721            26.93                21.78            22.56
 Credit Card           258        1281044          4965.29        349392            27.27                21.61            20.72
         UPI           252        1250473          4962.19        333889            26.70                21.11            20.23
         COD           206        1141790          5542.67        255744            22.40                17.25            18.47
         EMI           218        1114297          5111.45        295951            26.56                18.26            18.02

Key Insights:
- Best Revenue Generator: Debit Card with $1,395,035.00
- Best Profit Margin: Credit Card with 27.27% margin


## 6. Business Question 5: Regional Performance - Top States & Cities

**Question:** Which states and cities generate the highest revenue? How do regional performance metrics vary?

**SQL Concepts:** GROUP BY on multiple columns, Ranking with aggregation, Comparison analysis

This query analyzes regional sales performance at both state and city levels.

In [8]:
# Query 5: Top 10 States by Revenue
query5a = """
SELECT 
    State,
    COUNT(*) as Number_of_Orders,
    COUNT(DISTINCT CustomerName) as Unique_Customers,
    ROUND(SUM(Amount), 2) as Total_Revenue,
    ROUND(SUM(Profit), 2) as Total_Profit,
    ROUND(AVG(Amount), 2) as Avg_Order_Value,
    ROUND((SUM(Profit) / SUM(Amount) * 100), 2) as Profit_Margin_Percent
FROM sales
GROUP BY State
ORDER BY Total_Revenue DESC
LIMIT 10
"""

result5a = conn.execute(query5a).fetchall()
columns5a = ['State', 'Orders', 'Unique_Customers', 'Total_Revenue', 'Total_Profit', 'Avg_Order_Value', 'Profit_Margin_%']
df_result5a = pd.DataFrame(result5a, columns=columns5a)

print("TOP 10 STATES BY REVENUE:")
print("=" * 110)
print(df_result5a.to_string(index=False))

# Query 5b: Top 10 Cities by Revenue
print("\n\n")

query5b = """
SELECT 
    City,
    State,
    COUNT(*) as Number_of_Orders,
    COUNT(DISTINCT CustomerName) as Unique_Customers,
    ROUND(SUM(Amount), 2) as Total_Revenue,
    ROUND(SUM(Profit), 2) as Total_Profit,
    ROUND(AVG(Amount), 2) as Avg_Order_Value
FROM sales
GROUP BY City, State
ORDER BY Total_Revenue DESC
LIMIT 10
"""

result5b = conn.execute(query5b).fetchall()
columns5b = ['City', 'State', 'Orders', 'Unique_Customers', 'Total_Revenue', 'Total_Profit', 'Avg_Order_Value']
df_result5b = pd.DataFrame(result5b, columns=columns5b)

print("TOP 10 CITIES BY REVENUE:")
print("=" * 120)
print(df_result5b.to_string(index=False))

TOP 10 STATES BY REVENUE:
     State  Orders  Unique_Customers  Total_Revenue  Total_Profit  Avg_Order_Value  Profit_Margin_%
  New York     226               151        1130048        308506          5000.21            27.30
   Florida     200               134        1091174        308706          5455.87            28.29
California     218               142        1086436        278814          4983.65            25.66
     Texas     189               132        1011475        257780          5351.72            25.49
  Illinois     181               125         978738        240372          5407.39            24.56
      Ohio     180               122         884768        216519          4915.38            24.47



TOP 10 CITIES BY REVENUE:
         City      State  Orders  Unique_Customers  Total_Revenue  Total_Profit  Avg_Order_Value
      Orlando    Florida      77                46         452158        128125          5872.18
San Francisco California      84                53 

## 7. Business Question 6: Product Sub-Category Performance & Profitability

**Question:** Which product sub-categories are most profitable? How do revenue and profit metrics differ across sub-categories?

**SQL Concepts:** GROUP BY, Multiple aggregations, Comparative analysis, Ranking

This query provides detailed profitability analysis for each sub-category.

In [9]:
# Query 6: Product Sub-Category Performance
query6 = """
SELECT 
    Category,
    "Sub-Category",
    COUNT(*) as Number_of_Orders,
    ROUND(SUM(Amount), 2) as Total_Revenue,
    ROUND(SUM(Profit), 2) as Total_Profit,
    ROUND((SUM(Profit) / SUM(Amount) * 100), 2) as Profit_Margin_Percent,
    ROUND(AVG(Quantity), 2) as Avg_Quantity,
    ROUND(SUM(Amount) / SUM(Quantity), 2) as Price_per_Unit
FROM sales
GROUP BY Category, "Sub-Category"
ORDER BY Total_Profit DESC
LIMIT 15
"""

result6 = conn.execute(query6).fetchall()
columns6 = ['Category', 'Sub_Category', 'Orders', 'Total_Revenue', 'Total_Profit', 'Profit_Margin_%', 'Avg_Quantity', 'Price_per_Unit']
df_result6 = pd.DataFrame(result6, columns=columns6)

print("TOP 15 SUB-CATEGORIES BY PROFIT:")
print("=" * 140)
print(df_result6.to_string(index=False))
print("\nKey Insights:")
print(f"- Most Profitable Sub-Category: {df_result6.iloc[0]['Sub_Category']} with ${df_result6.iloc[0]['Total_Profit']:,.2f} profit")
print(f"- Highest Margin: {df_result6.loc[df_result6['Profit_Margin_%'].idxmax(), 'Sub_Category']} at {df_result6['Profit_Margin_%'].max():.2f}% margin")

TOP 15 SUB-CATEGORIES BY PROFIT:
       Category     Sub_Category  Orders  Total_Revenue  Total_Profit  Profit_Margin_%  Avg_Quantity  Price_per_Unit
Office Supplies          Markers     110         627875        174749            27.83         10.66          535.27
      Furniture           Tables     122         625177        156796            25.08         10.68          479.80
Office Supplies            Paper     104         524755        149723            28.53          9.43          534.92
    Electronics Electronic Games     104         565092        148454            26.27         11.73          463.19
    Electronics         Printers      95         566359        146259            25.82         11.83          503.88
      Furniture            Sofas     114         568367        142854            25.13         10.82          460.96
Office Supplies             Pens     114         552269        129846            23.51         10.56          458.70
      Furniture           Chair

## 8. Business Question 7: Customer Lifetime Value (CLV) & RFM Analysis

**Question:** Who are our most valuable customers? What are their lifetime purchase values and activity patterns (Recency, Frequency, Monetary)?

**SQL Concepts:** Window functions, Ranking, Multiple aggregations, Date calculations, CTEs

This query calculates customer lifetime value and creates RFM segments.

In [10]:
# Query 7: Top 20 Customers by Lifetime Value (CLV) with RFM Analysis
query7 = """
WITH customer_metrics AS (
    SELECT 
        CustomerName,
        State,
        COUNT(*) as Frequency,
        ROUND(SUM(Amount), 2) as Monetary,
        ROUND(SUM(Profit), 2) as Total_Profit,
        MAX(CAST("Order Date" AS DATE)) as Last_Purchase_Date,
        MIN(CAST("Order Date" AS DATE)) as First_Purchase_Date,
        ROUND(AVG(Amount), 2) as Avg_Order_Value
    FROM sales
    GROUP BY CustomerName, State
),
customer_ranked AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (ORDER BY Monetary DESC) as CLV_Rank,
        CASE 
            WHEN Frequency >= 10 THEN 'Champions'
            WHEN Frequency >= 5 AND Monetary > (SELECT AVG(Monetary) FROM customer_metrics) THEN 'Loyal Customers'
            WHEN Frequency >= 3 THEN 'Potential Loyalists'
            ELSE 'At-Risk/Inactive'
        END as Customer_Segment
    FROM customer_metrics
)
SELECT 
    CLV_Rank,
    CustomerName,
    State,
    Frequency,
    Monetary as Lifetime_Value,
    Total_Profit,
    Avg_Order_Value,
    Customer_Segment,
    Last_Purchase_Date
FROM customer_ranked
WHERE CLV_Rank <= 20
ORDER BY CLV_Rank
"""

result7 = conn.execute(query7).fetchall()
columns7 = ['CLV_Rank', 'Customer_Name', 'State', 'Frequency', 'Lifetime_Value', 'Total_Profit', 'Avg_Order_Value', 'Segment', 'Last_Purchase']
df_result7 = pd.DataFrame(result7, columns=columns7)

print("TOP 20 CUSTOMERS BY LIFETIME VALUE (CLV) WITH RFM SEGMENTATION:")
print("=" * 150)
print(df_result7.to_string(index=False))
print(f"\nTop Customer: {df_result7.iloc[0]['Customer_Name']} with Lifetime Value of ${df_result7.iloc[0]['Lifetime_Value']:,.2f}")
print(f"Total Revenue from Top 20 Customers: ${df_result7['Lifetime_Value'].sum():,.2f}")

TOP 20 CUSTOMERS BY LIFETIME VALUE (CLV) WITH RFM SEGMENTATION:
 CLV_Rank        Customer_Name      State  Frequency  Lifetime_Value  Total_Profit  Avg_Order_Value             Segment Last_Purchase
        1           Cory Evans    Florida          4           28557          7790          7139.25 Potential Loyalists    2024-06-16
        2        Emily Ellison California          3           27352          6848          9117.33 Potential Loyalists    2022-08-17
        3    Nicholas Anderson    Florida          3           27352          6848          9117.33 Potential Loyalists    2021-11-16
        4        George Foster      Texas          3           27352          6848          9117.33 Potential Loyalists    2022-06-10
        5   Katherine Williams    Florida          3           25121          5972          8373.67 Potential Loyalists    2022-05-30
        6        Randy Johnson       Ohio          3           24295          5858          8098.33 Potential Loyalists    2022-12-2

## 9. Summary & Key Takeaways

This notebook demonstrated 7 key business questions that can be answered using SQL queries:

1. **Top Product Categories by Revenue** - Identifies which product categories generate the most revenue and profit
2. **Monthly Revenue Trends** - Shows revenue patterns over time and growth metrics  
3. **Customer Segmentation** - Segments customers by purchase frequency for targeted marketing
4. **Payment Method Analysis** - Compares performance of different payment modes
5. **Regional Performance** - Identifies top-performing states and cities
6. **Sub-Category Profitability** - Ranks products by profitability and margins
7. **Customer Lifetime Value** - Identifies most valuable customers and segments them

**Key SQL Concepts Used:**
- **Filtering**: WHERE clauses to select specific data
- **Aggregation**: SUM(), COUNT(), AVG() to compute metrics
- **Grouping**: GROUP BY to segment data
- **Ordering**: ORDER BY and LIMIT to rank results
- **Window Functions**: LAG(), ROW_NUMBER() for advanced analytics
- **Conditional Logic**: CASE statements for segmentation
- **Common Table Expressions (CTEs)**: WITH clauses for complex queries
- **Date Functions**: Working with temporal data

These queries provide actionable insights for business decision-making!